In [0]:
import re
import pandas as pd
import numpy as np
import nltk
import string
import warnings

***Importing dataSet***

In [0]:
dataset=pd.read_csv('train.csv')
testdata=pd.read_csv('test.csv')

***Cleaning Tweets***

In [0]:
def cleanhtml(sentence):
    cleanr=re.compile('<.*?>')
    cleantext=re.sub(cleanr,' ',sentence)
    return cleantext

In [0]:
def cleanpunc(sentence):
    cleaned=re.sub(r'[?!\\\'"#]',r' ',sentence)
    cleaned=re.sub(r'[,)(\/]',r' ',cleaned)
    return cleaned

In [0]:
def r_pattern(txt):
  t=re.findall("@[\w]*",txt)
  for i in t:
    txt=re.sub(i,'',txt)
  return txt

In [249]:
tdata=dataset.append(testdata,ignore_index=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [0]:
tdata['tweet']=np.vectorize(r_pattern)(tdata['tweet'])

In [251]:
tdata['tweet']=tdata['tweet'].replace("[^a-zA-Z]#"," ")
tdata.head()

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so sel...
1,2,0.0,thanks for #lyft credit i can't use cause th...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation


Remove short words and stop word

In [0]:
tdata['tweet']=tdata['tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [0]:
token=tdata['tweet'].apply(lambda x: x.split())

In [0]:
from nltk.stem.porter import *
stemmer=PorterStemmer()

In [0]:
token=tdata['tweet'].apply(lambda x: [stemmer.stem(i) for i in x])

In [0]:
for i in range(len(token)):
  token[i]=''.join(token[i])

In [257]:
tdata['tweet']=token
tdata.head()

,id,label,tweet
0,1,0.0,when father dysfunctional selfish drags kids i...
1,2,0.0,thanks #lyft credit can't cause they don't off...
2,3,0.0,bihday your majesty
3,4,0.0,#model love take with time urð±!!! ððð...
4,5,0.0,factsguide: society #motivation


In [0]:
from sklearn.feature_extraction.text import CountVectorizer
bow_v=CountVectorizer(max_df=0.90,min_df=2,max_features=1000,stop_words='english')

In [0]:
bow=bow_v.fit_transform(tdata['tweet'])

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [0]:
train_b=bow[:31962,:]
test_b=bow[31962:,:]

In [0]:
xtrain_b,xvalid_b,ytrain,yvalid = train_test_split(train_b,dataset['label'],random_state=42,test_size=0.3)

In [0]:
from sklearn.svm import SVC

In [0]:
svm=SVC()
svm = SVC(kernel = 'rbf', random_state = 0, gamma = 0.14, C =11)

In [265]:
svm.fit(xtrain_b,ytrain)

SVC(C=11, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.14, kernel='rbf',
    max_iter=-1, probability=False, random_state=0, shrinking=True, tol=0.001,
    verbose=False)

In [266]:
y_pred=svm.predict(xvalid_b)
prediction_int=y_pred.astype(np.int)
f1_score(yvalid,prediction_int)

0.5340802987861811

Prediction on test set

In [0]:
test_pred=svm.predict(test_b)
test_pred_int=test_pred.astype(np.int)
testdata['label']=test_pred_int
submission=testdata[['id','label']]
submission.to_csv('svm_submission.csv',index=False)